# Compare partitioning methods

Will consider the following partitioning strategies, potentially coming up with better algorithms as well. 

1. Metis/ ParMetis
2. Scotch/PT-scotch
3. Fennel
4. SFC-based (Morton, Hilbert)
5. Pagerank
6. Heat-kernel 

We will evaluate these primarily in the context of FEM solvers, CG and DG high-order models, and will look at a large collection of Hex and Tet meshes, 10,000 each. Additional refinements will also be considered.

The key metrics for evaluation are, 

1. performance (time to solution)
2. scalability (how well can we scale up for large meshes)
3. Compute costs
    - load imbalance 
    - scalability of load balance
4. Communication costs
    - cost of cut (total communication)
    - max communication for any processor 
    - max number of processors communicated with
    - scalability of above quantities
5. FEM code runtime 
    - CG example
    - DG example 
    - Our code
    - `mfem`
 
Implement initial tests and implementations in Python, and later the parallel version in `C++`. Except for `Scotch` everything else should be easy to evaluate in `Python`. Scalability experiments will only be performed in `C++` with `MPI` and `OpenMP`. 


In [1]:
import sys
import glob

import gmsh
import networkx as nx
import metis

import pandas as pd

gmsh.initialize() #sys.argv)

We will store partition results in a `pandas` dataframe backed by an excel file. This can simplify data analysis and plotting later. Here, we use metrics $\lambda=|E_{\text{cut}}|/|E|$ and $\rho = N_{\max}/\frac{N}{p}$. 

In [2]:
try:
  results = pd.read_excel('partition.xlsx')
  index = set(results.columns[0])
except FileNotFoundError:
  results = pd.DataFrame(columns=['num_elems', 'num_faces', 'type', 'npes', 'metis_lambda', 'metis_rho', 'metis_loads', 'metis_comms'])
  index = set()

We will use the meshes from the [NYU geometric datasets](https://cims.nyu.edu/gcl/datasets.html) collection, available [here](https://archive.nyu.edu/handle/2451/44221) 



In [3]:
## Params 
np = 13
folder = r'Meshes/10k_tet/*.mesh'
stop_after = 200

In [4]:
num_files = 0

for fname in glob.glob(folder):
    gmsh.open(fname)

    if len(results) > 0 and gmsh.model.getCurrent() in index:
        gmsh.clear()
        continue
    
    if num_files > stop_after:
        break
    num_files += 1

    print(str(num_files) + ': Model ' + gmsh.model.getCurrent() + ' (' + str(gmsh.model.getDimension()) + 'D)')
    
    G = nx.Graph()

    entities = gmsh.model.getEntities()
    mesh_type = ''

    for e in entities:
        dim = e[0]
        tag = e[1]

    if dim != 3:
        continue
    
    type = gmsh.model.getType(e[0], e[1])
    name = gmsh.model.getEntityName(e[0], e[1])
    if len(name): name += ' '
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(dim, tag)

    if elemTypes[0] == 4:
        mesh_type = 'tet'
        fn = 3
        en = 12 
        elems, _ = gmsh.model.mesh.getElementsByType(4)
        faces    = gmsh.model.mesh.getElementFaceNodes(4, 3)
    else:
        mesh_type = 'hex'
        fn = 4
        en = 24
        elems, _ = gmsh.model.mesh.getElementsByType(5)
        faces    = gmsh.model.mesh.getElementFaceNodes(5, 4)

    # print('  Have ', len(elems), ' elements and ', len(faces), ' faces.')

    f2e = {}
    e2e = {}

    V = {}
    for i,x in enumerate(elems):
        v = G.add_node(x)
        V[i] = x

    for i in range(0, len(faces), fn):
        f = tuple(sorted(faces[i:i+fn]))
        t = elems[i//en]
        if not f in f2e:
            f2e[f] = [t]
        else:
            f2e[f].append(t)

    # compute neighbors by face
    for i in range(0, len(faces), fn):
        f = tuple(sorted(faces[i:i+fn]))
        t = elems[i//en]
        if not t in e2e:
            e2e[t] = set()
        for tt in f2e[f]:
            if tt != t:
                e2e[t].add(tt)

    for k in e2e:
        for j in e2e[k]:
            G.add_edge(k,j)
        
    (edgecuts, parts) = metis.part_graph(G, np)

    parts2 = {}

    for i,p in enumerate(parts):
        parts2[V[i]] = p

    ###------------ metrics ------------###

    part_elems = [0] * np 
    part_sends = [0] * np

    for e in elems:
        part_elems[parts2[e]] += 1

    for k in e2e:
        for j in e2e[k]:
            if parts2[k] != parts2[j]:
                # send k - j
                part_sends[parts2[k]] += 1

    lam = edgecuts/len(faces)
    rho = max(part_elems)/(len(elems)/np) 


    if len(results) == 0 or gmsh.model.getCurrent() not in index:
        # 'num_elems', 'num_faces', 'type', 'npes', 'metis_lambda', 'metis_rho', 'metis_loads', 'metis_comms'
        s = pd.Series({'num_elems':len(elems), 'num_faces':len(faces), 'type':mesh_type, 'npes':np, 'metis_lambda':lam, 'metis_rho':rho, 'metis_loads':part_elems, 'metis_comms':part_sends})
        s.name = gmsh.model.getCurrent()
        index.add(gmsh.model.getCurrent())
        results = results.append(s)


results.to_excel("partition.xlsx")
print("Total results: %s" % len(results))
print("")


Info    : Reading 'Meshes/10k_tet/1454018_sf_hexa.mesh_15478_80771.obj.mesh'...
Info    : 80586 nodes
Info    : 0 triangles
Info    : 418211 tetrahedra
Info    : Done reading 'Meshes/10k_tet/1454018_sf_hexa.mesh_15478_80771.obj.mesh'
Model 1454018_sf_hexa.mesh_15478_80771.obj (3D)
Info    : Reading 'Meshes/10k_tet/289691_sf_hexa.mesh_14442_62100.obj.mesh'...
Info    : 61883 nodes
Info    : 0 triangles
Info    : 304902 tetrahedra
Model 289691_sf_hexa.mesh_14442_62100.obj (3D)Info    : Done reading 'Meshes/10k_tet/289691_sf_hexa.mesh_14442_62100.obj.mesh'

Info    : Reading 'Meshes/10k_tet/186553_sf_hexa.mesh_15608_84019.obj.mesh'...
Info    : 84019 nodes
Info    : 0 triangles
Info    : 439917 tetrahedra
Info    : Done reading 'Meshes/10k_tet/186553_sf_hexa.mesh_15608_84019.obj.mesh'
Model 186553_sf_hexa.mesh_15608_84019.obj (3D)
Info    : Reading 'Meshes/10k_tet/940040_sf_hexa.mesh_44818_160759.obj.mesh'...
Info    : 160041 nodes
Info    : 0 triangles
Info    : 811391 tetrahedra
Model 9

Model 106784_sf_hexa.mesh_23690_88173.obj (3D)Info    : Done reading 'Meshes/10k_tet/106784_sf_hexa.mesh_23690_88173.obj.mesh'

Info    : Reading 'Meshes/10k_tet/1722415_sf_hexa.mesh_7552_33844.obj.mesh'...
Info    : 33734 nodes
Info    : 0 triangles
Info    : 172772 tetrahedra
Model 1722415_sf_hexa.mesh_7552_33844.obj (3D)Info    : Done reading 'Meshes/10k_tet/1722415_sf_hexa.mesh_7552_33844.obj.mesh'

Info    : Reading 'Meshes/10k_tet/204957_sf_hexa.mesh_13422_53009.obj.mesh'...
Info    : 51614 nodes
Info    : 0 triangles
Info    : 245526 tetrahedra
Model 204957_sf_hexa.mesh_13422_53009.obj (3D)Info    : Done reading 'Meshes/10k_tet/204957_sf_hexa.mesh_13422_53009.obj.mesh'

Info    : Reading 'Meshes/10k_tet/96074_sf_hexa.mesh_30926_108729.obj.mesh'...
Info    : 104515 nodes
Info    : 0 triangles
Info    : 518005 tetrahedra
Model 96074_sf_hexa.mesh_30926_108729.obj (3D)Info    : Done reading 'Meshes/10k_tet/96074_sf_hexa.mesh_30926_108729.obj.mesh'

Info    : Reading 'Meshes/10k_tet/

Info    : Reading 'Meshes/10k_tet/472112_sf_hexa.mesh_5040_23157.obj.mesh'...
Info    : 23157 nodes
Info    : 0 triangles
Info    : 116652 tetrahedra
Info    : Done reading 'Meshes/10k_tet/472112_sf_hexa.mesh_5040_23157.obj.mesh'
Model 472112_sf_hexa.mesh_5040_23157.obj (3D)
Info    : Reading 'Meshes/10k_tet/53118_sf_hexa.mesh_13472_58322.obj.mesh'...
Info    : 57323 nodes
Info    : 0 triangles
Info    : 286495 tetrahedra
Info    : Done reading 'Meshes/10k_tet/53118_sf_hexa.mesh_13472_58322.obj.mesh'
Model 53118_sf_hexa.mesh_13472_58322.obj (3D)
Info    : Reading 'Meshes/10k_tet/83835_sf_hexa.mesh_17956_69439.obj.mesh'...
Info    : 68579 nodes
Info    : 0 triangles
Info    : 349033 tetrahedra
Info    : Done reading 'Meshes/10k_tet/83835_sf_hexa.mesh_17956_69439.obj.mesh'
Model 83835_sf_hexa.mesh_17956_69439.obj (3D)
Info    : Reading 'Meshes/10k_tet/110445_sf_hexa.mesh_9856_44137.obj.mesh'...
Info    : 44022 nodes
Info    : 0 triangles
Info    : 214289 tetrahedra
Info    : Done reading

Info    : Reading 'Meshes/10k_tet/45813_sf_hexa.mesh_10712_39055.obj.mesh'...
Info    : 38981 nodes
Info    : 0 triangles
Info    : 190455 tetrahedra
Info    : Done reading 'Meshes/10k_tet/45813_sf_hexa.mesh_10712_39055.obj.mesh'
Model 45813_sf_hexa.mesh_10712_39055.obj (3D)
Info    : Reading 'Meshes/10k_tet/96073_sf_hexa.mesh_29332_100811.obj.mesh'...
Info    : 96054 nodes
Info    : 0 triangles
Info    : 475174 tetrahedra
Info    : Done reading 'Meshes/10k_tet/96073_sf_hexa.mesh_29332_100811.obj.mesh'
Model 96073_sf_hexa.mesh_29332_100811.obj (3D)
Info    : Reading 'Meshes/10k_tet/384086_sf_hexa.mesh_11736_41960.obj.mesh'...
Info    : 41600 nodes
Info    : 0 triangles
Info    : 208725 tetrahedra
Info    : Done reading 'Meshes/10k_tet/384086_sf_hexa.mesh_11736_41960.obj.mesh'
Model 384086_sf_hexa.mesh_11736_41960.obj (3D)
Info    : Reading 'Meshes/10k_tet/85580_sf_hexa.mesh_17918_135659.obj.mesh'...
Info    : 135659 nodes
Info    : 0 triangles
Info    : 719042 tetrahedra
Info    : Done

Info    : Done reading 'Meshes/10k_tet/697203_sf_hexa.mesh_23384_113767.obj.mesh'
Model 697203_sf_hexa.mesh_23384_113767.obj (3D)
Info    : Reading 'Meshes/10k_tet/66611_sf_hexa.mesh_22130_77313.obj.mesh'...
Info    : 75888 nodes
Info    : 0 triangles
Info    : 376634 tetrahedra
Info    : Done reading 'Meshes/10k_tet/66611_sf_hexa.mesh_22130_77313.obj.mesh'
Model 66611_sf_hexa.mesh_22130_77313.obj (3D)
Info    : Reading 'Meshes/10k_tet/48340_sf_hexa.mesh_13120_52924.obj.mesh'...
Info    : 52372 nodes
Info    : 0 triangles
Info    : 264734 tetrahedra
Info    : Done reading 'Meshes/10k_tet/48340_sf_hexa.mesh_13120_52924.obj.mesh'
Model 48340_sf_hexa.mesh_13120_52924.obj (3D)
Info    : Reading 'Meshes/10k_tet/1224383_sf_hexa.mesh_25806_104739.obj.mesh'...
Info    : 104231 nodes
Info    : 0 triangles
Info    : 535340 tetrahedra
Info    : Done reading 'Meshes/10k_tet/1224383_sf_hexa.mesh_25806_104739.obj.mesh'
Model 1224383_sf_hexa.mesh_25806_104739.obj (3D)
Info    : Reading 'Meshes/10k_te

Info    : Reading 'Meshes/10k_tet/61082_sf_hexa.mesh_9646_39073.obj.mesh'...
Info    : 38937 nodes
Info    : 0 triangles
Info    : 196562 tetrahedra
Info    : Done reading 'Meshes/10k_tet/61082_sf_hexa.mesh_9646_39073.obj.mesh'
Model 61082_sf_hexa.mesh_9646_39073.obj (3D)
Info    : Reading 'Meshes/10k_tet/1146191_sf_hexa.mesh_44944_219368.obj.mesh'...
Info    : 213535 nodes
Info    : 0 triangles
Info    : 1096983 tetrahedra
Info    : Done reading 'Meshes/10k_tet/1146191_sf_hexa.mesh_44944_219368.obj.mesh'
Model 1146191_sf_hexa.mesh_44944_219368.obj (3D)
Info    : Reading 'Meshes/10k_tet/472054_sf_hexa.mesh_29128_99915.obj.mesh'...
Info    : 99071 nodes
Info    : 0 triangles
Info    : 493501 tetrahedra
Info    : Done reading 'Meshes/10k_tet/472054_sf_hexa.mesh_29128_99915.obj.mesh'
Model 472054_sf_hexa.mesh_29128_99915.obj (3D)
Info    : Reading 'Meshes/10k_tet/113862_sf_hexa.mesh_12534_43792.obj.mesh'...
Info    : 43741 nodes
Info    : 0 triangles
Info    : 203355 tetrahedra
Info    : 

Info    : Done reading 'Meshes/10k_tet/225969_sf_hexa.mesh_11502_43513.obj.mesh'
Model 225969_sf_hexa.mesh_11502_43513.obj (3D)
Info    : Reading 'Meshes/10k_tet/106785_sf_hexa.mesh_30834_117576.obj.mesh'...
Info    : 113311 nodes
Info    : 0 triangles
Info    : 564600 tetrahedra
Info    : Done reading 'Meshes/10k_tet/106785_sf_hexa.mesh_30834_117576.obj.mesh'
Model 106785_sf_hexa.mesh_30834_117576.obj (3D)
Info    : Reading 'Meshes/10k_tet/1036317_sf_hexa.mesh_12356_49789.obj.mesh'...
Info    : 49763 nodes
Info    : 0 triangles
Info    : 249575 tetrahedra
Info    : Done reading 'Meshes/10k_tet/1036317_sf_hexa.mesh_12356_49789.obj.mesh'
Model 1036317_sf_hexa.mesh_12356_49789.obj (3D)
Info    : Reading 'Meshes/10k_tet/491062_sf_hexa.mesh_31078_149243.obj.mesh'...
Info    : 149243 nodes
Info    : 0 triangles
Info    : 781906 tetrahedra
Info    : Done reading 'Meshes/10k_tet/491062_sf_hexa.mesh_31078_149243.obj.mesh'
Model 491062_sf_hexa.mesh_31078_149243.obj (3D)
Info    : Reading 'Meshe

In [ ]:
###------------ Clean up ------------###
gmsh.clear()
gmsh.finalize()